In [4]:
## threading 模块多线程 API，Thread 是线程类，提供了线程相关函数
## active_count(): 当前处于活动状态的线程个数
## current_thread(): 返回当前线程对象
## main_thread(): 返回主线程对象，主线程对象是由 Python 解释器提供的线程

import threading

current_thread = threading.current_thread()
print('当前线程名称 => ' + current_thread.getName())

activeCount = threading.active_count()
print('当前活动线程数量 => ', activeCount)

mainThread = threading.main_thread()
print('主线程名称 => ' + mainThread.getName())


当前线程名称 => MainThread
当前活动线程数量 =>  8
主线程名称 => MainThread


In [14]:

import threading
import time

## 线程体：线程体是子线程要执行的代码
def thread_body():
	t = threading.current_thread()
	time.sleep(2)
	print('线程: {0} 执行完成'.format(t.getName()))

## 线程对象：线程对象是 threading 模块的线程类 Thread 或 Thread 子类所创建的对象
t1 = threading.Thread(target=thread_body, name="线程1")
## 启动线程
t1.start()

print('线程 {0} 执行完成\n'.format(threading.current_thread().getName()))

线程 MainThread 执行完成



线程: 线程1 执行完成


In [15]:
import threading
import time

## 通过 Thread 子类创建线程
class SmallThread(threading.Thread):
	def __init__(self, name=None):
		super().__init__(name=name)

	## 重写 Thread run 函数
	def run(self):
		t = threading.current_thread()
		time.sleep(2)
		print('线程 {0} 执行完毕'.format(t.getName()))

t1 = SmallThread(name='MyThread')
t1.start()

线程 MyThread 执行完毕


In [3]:
import threading
import time

def thread_body():
	time.sleep(2)
	print('线程 {0} 执行完毕'.format(threading.current_thread().getName()))

t = threading.Thread(target=thread_body, name='Thread1')
t.start()

## 阻塞主线程，等待 Thread1 线程执行完毕后，才往后执行
t.join()

print('主线程执行完毕')

线程 Thread1 执行完毕
主线程执行完毕


In [4]:
from concurrent.futures import ThreadPoolExecutor
import threading
import random
import time

def task() -> int:
	sleep_time = random.randint(1, 5)
	print(f'线程名称：{threading.current_thread().name}，睡眠时间: {sleep_time}')
	time.sleep(sleep_time)
	return sleep_time

# 创建线程池，最大工作线程为 2
with ThreadPoolExecutor(max_workers=2) as executor:
	for _ in range(5):
		future = executor.submit(task)
		# 判断线程是否执行结束
		if future.done():
			print(f'执行结果: {future.result()}')

线程名称：ThreadPoolExecutor-3_0，睡眠时间: 3
线程名称：ThreadPoolExecutor-3_1，睡眠时间: 2
线程名称：ThreadPoolExecutor-3_1，睡眠时间: 4
线程名称：ThreadPoolExecutor-3_0，睡眠时间: 1
线程名称：ThreadPoolExecutor-3_0，睡眠时间: 2


In [6]:
from concurrent.futures import wait, FIRST_COMPLETED

with ThreadPoolExecutor(max_workers=2) as executor:
	# 任务队列
	wait_list = [executor.submit(task) for _ in range(5)]

  #  FIRST_COMPLETED 表示 完成第一个就停止等待
	# wait(fs, timeout=None, return_when=ALL_COMPLETED)
	wait(wait_list) # wait 等待队列中的任务完成
	print('程序执行完成')
	

线程名称：ThreadPoolExecutor-5_0，睡眠时间: 2
线程名称：ThreadPoolExecutor-5_1，睡眠时间: 3
线程名称：ThreadPoolExecutor-5_0，睡眠时间: 2
线程名称：ThreadPoolExecutor-5_1，睡眠时间: 3
线程名称：ThreadPoolExecutor-5_0，睡眠时间: 3
程序执行完成


In [9]:
# 当某个任务完成后，就将结果交由主线程，而不是通过主线程对每个线程都进行完成判断
from concurrent.futures import as_completed

with ThreadPoolExecutor(max_workers=2) as executor:
	task_list = [executor.submit(task) for _ in range(5)]
	for future in as_completed(task_list):
		print(f'线程执行结果：{future.result()}')
	print('程序执行完成')

线程名称：ThreadPoolExecutor-8_0，睡眠时间: 1
线程名称：ThreadPoolExecutor-8_1，睡眠时间: 3
线程名称：ThreadPoolExecutor-8_0，睡眠时间: 3线程执行结果：1

线程名称：ThreadPoolExecutor-8_1，睡眠时间: 1
线程执行结果：3
线程名称：ThreadPoolExecutor-8_0，睡眠时间: 1线程执行结果：3

线程执行结果：1
线程执行结果：1
程序执行完成


In [12]:

def task_map(i):
	sleep_time = random.randint(1, 5)
	print(f'{threading.current_thread().name}, 参数: {i}')
	time.sleep(sleep_time)

with ThreadPoolExecutor(max_workers=2) as executor:
	# map 方法与 python 高阶函数 map 的含义相同，都是将序列中的每个元素都执行同一个函数 
	for result in executor.map(task_map, [1, 2, 3, 4, 5]):
		print(result)
		

ThreadPoolExecutor-11_0, 参数: 1
ThreadPoolExecutor-11_1, 参数: 2
ThreadPoolExecutor-11_0, 参数: 3ThreadPoolExecutor-11_1, 参数: 4
None
None

ThreadPoolExecutor-11_1, 参数: 5
None
None
None


### 使用多线程实现交替打印 A、B、C 10次

In [9]:
import threading

def print_a():
    for _ in range(10):
        semaphoreA.acquire() # 锁定自己的信号量
        print(f'{threading.current_thread().name} -  A')
        semaphoreB.release() # 释放下一个打印的信号量

def print_b():
    for _ in range(10):
        semaphoreB.acquire()
        print(f'{threading.current_thread().name} -  B')
        semaphoreC.release()

def print_c():
    for _ in range(10):
        semaphoreC.acquire()
        print(f'{threading.current_thread().name} -  C')
        semaphoreA.release()

# 创建信号量
semaphoreA = threading.Semaphore(1)
semaphoreB = threading.Semaphore(0)
semaphoreC = threading.Semaphore(0)

# 创建线程
threadA = threading.Thread(target=print_a, name='ThreadA')
threadB = threading.Thread(target=print_b, name='ThreadB')
threadC = threading.Thread(target=print_c, name='ThreadC')

# 启动线程
threadA.start()
threadB.start()
threadC.start()

ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B


ThreadC -  C


In [11]:
import threading

def print_a():
    for _ in range(10):
        lockC.acquire() # 获取上一个打印的锁
        print(f'{threading.current_thread().name} -  A')
        lockA.release() # 释放自己的锁
        
def print_b():
    for _ in range(10):
        lockA.acquire()
        print(f'{threading.current_thread().name} -  B')
        lockB.release()
        
def print_c():
    for _ in range(10):
        lockB.acquire()
        print(f'{threading.current_thread().name} -  C')
        lockC.release()

# 创建锁
lockA = threading.Lock()
lockB = threading.Lock()
lockC = threading.Lock()

# 创建线程
threadA = threading.Thread(target=print_a, name='ThreadA')
threadB = threading.Thread(target=print_b, name='ThreadB')
threadC = threading.Thread(target=print_c, name='ThreadC')

# 先锁 A、B 保证 线程A 最先输出
lockA.acquire()
lockB.acquire()

# 启动线程
threadA.start()
threadB.start()
threadC.start()



ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A
ThreadB -  B
ThreadC -  C
ThreadA -  A


ThreadB -  B
ThreadC -  C
